# Calculating NDVI

Normalized Digital Vegetation Index [(NDVI)](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) is an important metric for analyzing plant health. In this tutorial, we will analyze the NDVI of the area surrounding Elkton, Virginia.

We make the usual imports and declarations.

In [1]:
import astraea.spark.rasterframes._
import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import geotrellis.spark._
import geotrellis.spark.io._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

implicit val spark = SparkSession.builder().
  master("local[*]").appName("RasterFrames").getOrCreate().withRasterFrames
spark.sparkContext.setLogLevel("ERROR")
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://172.18.0.2:4041
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1532095243432)
SparkSession available as 'spark'


import astraea.spark.rasterframes._
import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import geotrellis.spark._
import geotrellis.spark.io._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@ab0d931
import spark.implicits._


In [2]:
val scene = SinglebandGeoTiff("samples/L8-B8-Robinson-IL.tiff")
val rf = scene.projectedRaster.toRF(128, 128).cache()

scene: geotrellis.raster.io.geotiff.SinglebandGeoTiff = SinglebandGeoTiff(geotrellis.raster.UShortConstantNoDataArrayTile@77b8f3df,Extent(431902.5, 4313647.5, 443512.5, 4321147.5),EPSG:32616,Tags(Map(AREA_OR_POINT -> POINT),List(Map())),GeoTiffOptions(geotrellis.raster.io.geotiff.Striped@6fb8338e,geotrellis.raster.io.geotiff.compression.DeflateCompression$@5fae03ad,1,None))
rf: org.apache.spark.sql.DataFrame with shapeless.tag.Tagged[astraea.spark.rasterframes.RasterFrameTag] = [spatial_key: struct<col: int, row: int>, tile: rf_tile]


Here's an example of computing the Normalized Differential Vegetation Index (NDVI) is a 
standardized vegetation index which allows us to generate an image highlighting differences in
relative biomass. 

> “An NDVI is often used worldwide to monitor drought, monitor and predict agricultural production, assist in predicting hazardous fire zones, and map desert encroachment. The NDVI is preferred for global vegetation monitoring because it helps to compensate for changing illumination conditions, surface slope, aspect, and other extraneous factors” (Lillesand. *Remote sensing and image interpretation*. 2004).

NDVI is the normalized difference of the nir and red bands:
$$ NDVI = \frac{nir - red}{nir + red}$$

In [3]:
def redBand = SinglebandGeoTiff("samples/L8-B4-Elkton-VA.tiff").projectedRaster.toRF("red_band")
def nirBand = SinglebandGeoTiff("samples/L8-B5-Elkton-VA.tiff").projectedRaster.toRF("nir_band")

// Define UDF for computing NDVI from red and NIR bands
val ndvi = udf((red: Tile, nir: Tile) ⇒ {
  val redd = red.convert(DoubleConstantNoDataCellType)
  val nird = nir.convert(DoubleConstantNoDataCellType)
  (nird - redd)/(nird + redd)
})

// We use `asRF` to indicate we know the structure still conforms to RasterFrame constraints
val rf = redBand.spatialJoin(nirBand).withColumn("ndvi", ndvi($"red_band", $"nir_band")).asRF

redBand: astraea.spark.rasterframes.RasterFrame
nirBand: astraea.spark.rasterframes.RasterFrame
ndvi: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function2>,org.apache.spark.sql.gt.types.TileUDT@58285bf1,Some(List(org.apache.spark.sql.gt.types.TileUDT@58285bf1, org.apache.spark.sql.gt.types.TileUDT@58285bf1)))
rf: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, red_band: rf_tile ... 2 more fields]


In [4]:
rf.select(tileStats($"ndvi")).show()

+---------+-----------+--------------------+------------------+-------------------+--------------------+
|dataCells|noDataCells|                 min|               max|               mean|            variance|
+---------+-----------+--------------------+------------------+-------------------+--------------------+
|    31433|         -1|-0.12488999119929595|0.6699834571985877|0.45558948033745933|0.010907294541915387|
+---------+-----------+--------------------+------------------+-------------------+--------------------+



In [5]:
val pr = rf.toRaster($"ndvi", 466, 428)

val brownToGreen = ColorRamp(
  RGBA(166,97,26,255),
  RGBA(223,194,125,255),
  RGBA(245,245,245,255),
  RGBA(128,205,193,255),
  RGBA(1,133,113,255)
).stops(128)

val colors = ColorMap.fromQuantileBreaks(pr.tile.histogramDouble(), brownToGreen)
// change writing location
pr.tile.color(colors).renderPng().write("outputs/rf-ndvi.png")

//For a georefrenced singleband greyscale image, could do: `GeoTiff(pr).write("ndvi.tiff")`

pr: geotrellis.raster.ProjectedRaster[geotrellis.raster.Tile] = ProjectedRaster(Raster(CroppedTile(DoubleConstantNoDataArrayTile([D@39a0d41,466,428),GridBounds(0,0,465,427)),Extent(703986.502389, 4249551.61978, 709549.093643, 4254601.8671)),utm-CS)
brownToGreen: geotrellis.raster.render.ColorRamp = geotrellis.raster.render.ColorRamp@199b035c
colors: geotrellis.raster.render.ColorMap = geotrellis.raster.render.DoubleColorMap@51b25796
